In [1]:
from selenium import webdriver
import datetime
import time
from time import sleep
import argparse
import os
import requests
import dill
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import random
import pandas as pd
import urllib.request
import unidecode

In [2]:
# Sleep list for after browser loaded
loading_sleep = [7,8,9,10]
# Sleep list for downloading posters
store_image_sleep = [0.1,0.2,0.3] # Worked for [0.5,1]
# Sleep list for going to next page
next_page_sleep = [5,7,8,10]
# List of all genres available on IMDB
genres= ['Action','Adventure','Animation','Biography','Comedy','Crime','Drama','Family','Fantasy','History',\
         'Horror','Musical','Mystery','Romance','Sci-Fi','Sport','Thriller','War','Western']

genre_list = ['Western']
start_num = 6001 # Movie number to start search on
max_pages = 24 # This is the number of pages to go through before saving dataframe
num_res_per_page = 250 # Choose 50 or 250


In [3]:
###### COMPLETED ##########
# Bryan Mac has (1-6000) for Action, Adventure, Animation, Biography, Comedy, Crime, Drama, Family, Fantasy,
#                            History, Horror, Musical, Mystery, Romance, Sci-Fi, Sport, Thriller, War, Western
# Bryan PC has (6001-12000) for Action, Adventure, Animation, Biography, Comedy, Crime, Drama, Family, Fantasy,
#                               History, Horror, Musical, Mystery, Romance, Sci-Fi, Sport, Thriller, War, ...

###### CATEGORY DETAILS ###########
# Action has 44,230
# Adventure has 21,924
# Animation has 6,247
# Biography has 6,357
# Comedy has 88,928
# Crime has 29,671
# Drama has 182,244
# Family has 13,830
# Fantasy has 13,396
# History has 7,184
# Horror has 27,973
# Musical has 9,407
# Mystery has 14,752
# Romance has 43,713
# Sci-Fi has 13,023
# Sport has 4,712
# Thriller has 41,818
# War has 8,732
# Western has 8,414

In [4]:
# Path to which we save the downloaded posters
dir_path_base='/Users/bryan/Box Sync/CSE496-final/posters'
#creates directory if not already there
if not os.path.exists(dir_path_base):
    os.mkdir(dir_path_base)

In [5]:
# Definition to download and save posters
def store_image(dir_path,link,name,year):
    sleep(random.choice(store_image_sleep))
    
    try_count = 0
    while try_count <=20: # Try 20 times
        try_count += 1
        try:
            response=requests.get(link)
            soup = BeautifulSoup(response.text, "html.parser")
            image = soup.find('div', class_ = "poster")
            file_name = name+'_'+year+'.jpg'
            urllib.request.urlretrieve(image.img['src'],os.path.join(dir_path, file_name))
            return 1
        except:
            continue
    # Returned if movie never able to be downloaded
    return 0

In [6]:
# Main definition for scraping pages
def get_links_by_genre(genre_in, start_num, num_res_per_page, max_pages):
    genre_print = genre_in.upper()
    
    dir_path= dir_path_base + '/' + genre_in + '-' + str(start_num) + '-' + str(num_res_per_page)
    
    #creates directory if not already there
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)
    
    '''Driver code for getting all image links by genre'''
    
    # Define options for the browser
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    options.set_headless()
    
    # Initialize empty listss
    title=[]
    rating=[]
    meta_score=[]
    year=[]
    votes=[]
    genre=[]
    links = []
    page_num = 1
    
    # Initialize movie number to start on
    curr_movie_num = start_num
    
    while page_num<=max_pages:
        
        print('...... %s SEARCH PAGE (%i out of %i) ......' %(genre_print,page_num,max_pages))
        
        # Define URL for this page
        url='https://www.imdb.com/search/title/?title_type=feature&genres='+genre_in.lower()+'&count='+str(num_res_per_page)+'&start='+str(curr_movie_num)+'&explore=genres&ref_=adv_nxt'
        browser = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver',chrome_options=options)
        #scrapes the very first page into the soup#
        
        # Below allows us to automate searching pages which do not exist
        try:
            browser.get(url)
        except:
            continue
            
        # Pause to give the broswer time to load any animations or ads
        sleep(random.choice(loading_sleep))
        soup = BeautifulSoup(browser.page_source,"html.parser")
        
        # Update current page number for next time
        curr_movie_num += num_res_per_page
    
        movie_all_page=soup.find_all('div',class_='lister-item mode-advanced')
        for item in movie_all_page:
            
            # scrape the name
            name=item.h3.a.text
            
            # Translate to closest unicode version of title name
            name = unidecode.unidecode(name)
            
            # Sanitize name before continuing
            name = name.replace(r'/','_')
            name = name.replace(r':','_')
            name = name.replace(r'\\','_')
            name = name.replace(r'*','_')
            name = name.replace(r'?','_')
            name = name.replace(r'<','_')
            name = name.replace(r'>','_')
            name = name.replace(r"'",'_')
            name = name.replace(r'!','_')
            
            # Scrape the rating
            if item.find('span',class_='value') is not None:
                imdb_rating = item.find('span',class_='value').text
            else: # If not rating on IMDB the movie has not yet released
                continue
    
            # Scrape the year
            curr_year = item.h3.find('span', class_ = 'lister-item-year').text[-5:-1]
            no_year = False
            if not curr_year:
                no_year = True
            
            # Run only if there is a year
            if no_year == False:
                # Skip movie if released starting in 2020 or year given not integer
                try:
                    if int(curr_year) > 2019:
                        continue
                except:
                    continue
                #scrape url of image,then download and store it
                saved_bool = store_image(dir_path,str('https://www.imdb.com'+item.h3.a['href']),name,curr_year)
            else:
                saved_bool = 0
            
            # Only append any of the below if image was able to be scraped
            if saved_bool == 1:
                
                # Saves url for each movie's page
                links.append(str('https://www.imdb.com'+item.h3.a['href']))
                
                rating.append(imdb_rating)
                    
                #if the movie has a meta score only then include it.Eliminates pre and post production movies
                #append meta-score
                if item.find('div', class_ = 'ratings-metascore') is not None:
                    curr_meta_score = item.find('div',class_='ratings-metascore').text.split()[0]
                    meta_score.append(curr_meta_score)
                else:
                    meta_score.append('NaN')
                # Append the name
                title.append(name)
                # Append year of current movie
                year.append(curr_year)
                
                # Scrape the number of votes
                if item.find('span',attrs={'name':'nv'}) is not None:
                    try:
                        votes.append(int(item.find('span',attrs={'name':'nv'})['data-value']))
                    except:
                        votes.append('NaN')
                else:
                    votes.append('NaN')
                    
                # Finally store genres (Note we split on comma and strip white space)
                genre.append([x.strip() for x in item.find('span',class_='genre').text.split(',')])
                
        #close old browser and pause for the set pause time
        browser.quit()
        sleep(random.choice(next_page_sleep))
        
        page_num += 1
        
    
    #convert to pandas data frame and store to csv
    movie_ratings = pd.DataFrame({'movie': title, 'year': year, 'imdb': rating, 'metascore': meta_score,
                                  'votes': votes, 'genre':genre, 'link':links})
    print('ALL SEARCH PAGES COMPLETED FOR CHOSEN GENRE')
    
    return movie_ratings
 

In [7]:
# Run over chosen genres in list

for genre in genre_list:
    t0 = time.time()
    movie_ratings = get_links_by_genre(genre, start_num, num_res_per_page, max_pages)
    t1 = time.time()

    print('It took %i minutes to complete %s' %(int((t1-t0)/60),genre))
    print('')

    # Save to csv
    movie_ratings.to_csv(dir_path_base + r'/' + genre + '-' + str(start_num) + '-' + str(num_res_per_page) + '.csv', index=False)
    


/Users/bryan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: use setter for headless property instead of set_headless
  app.launch_new_instance()
/Users/bryan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: DeprecationWarning: use options instead of chrome_options


...... WESTERN SEARCH PAGE (1 out of 24) ......
...... WESTERN SEARCH PAGE (2 out of 24) ......
...... WESTERN SEARCH PAGE (3 out of 24) ......
...... WESTERN SEARCH PAGE (4 out of 24) ......
...... WESTERN SEARCH PAGE (5 out of 24) ......
...... WESTERN SEARCH PAGE (6 out of 24) ......
...... WESTERN SEARCH PAGE (7 out of 24) ......
...... WESTERN SEARCH PAGE (8 out of 24) ......
...... WESTERN SEARCH PAGE (9 out of 24) ......
...... WESTERN SEARCH PAGE (10 out of 24) ......
...... WESTERN SEARCH PAGE (11 out of 24) ......
...... WESTERN SEARCH PAGE (12 out of 24) ......
...... WESTERN SEARCH PAGE (13 out of 24) ......
...... WESTERN SEARCH PAGE (14 out of 24) ......
...... WESTERN SEARCH PAGE (15 out of 24) ......
...... WESTERN SEARCH PAGE (16 out of 24) ......
...... WESTERN SEARCH PAGE (17 out of 24) ......
...... WESTERN SEARCH PAGE (18 out of 24) ......
...... WESTERN SEARCH PAGE (19 out of 24) ......
...... WESTERN SEARCH PAGE (20 out of 24) ......
...... WESTERN SEARCH PAGE (2